# Initial Epochs

## Import 

In [ ]:
import pathlib
import matplotlib

import mne
import os
import mne_bids
import matplotlib.pyplot as plt

from autoreject import Ransac  
from autoreject.utils import (interpolate_bads, set_matplotlib_defaults) 

matplotlib.use('Agg') #Raises no browsers
#matplotlib.use('Qt5Agg') #Raises browsers

## Input Recording 

The Raw data structure: continuous data
https://mne.tools/stable/auto_tutorials/raw/10_raw_overview.html

In [ ]:
bids_path = mne_bids.BIDSPath(subject=subject,
                              session=session,
                              task=task,
                              datatype='eeg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path) 
raw.load_data() 
events, event_id = mne.events_from_annotations(raw) 

## Reset Channel Types

The Info data structure
https://mne.tools/stable/auto_tutorials/intro/30_info.html

In [ ]:
raw = raw.copy().pick_types(meg=False, eeg=True, eog=True)
raw.set_channel_types({'E8': 'eog'})
raw.set_channel_types({'E25': 'eog'})
len(raw.ch_names)
raw

Background on projectors and projections
https://mne.tools/stable/auto_tutorials/preprocessing/45_projectors_background.html

Setting the EEG reference
https://mne.tools/stable/auto_tutorials/preprocessing/55_setting_eeg_reference.html

In [ ]:
raw.set_eeg_reference('average',ch_type='eeg',projection=True)  # needed for inverse modeling

## Filter Data

Filtering and resampling data
https://mne.tools/stable/auto_tutorials/preprocessing/30_filtering_resampling.html

In [ ]:
filter_raw = raw.filter(l_freq=0.1, h_freq=40) 
filter_raw

## Epoching Data 

The Epochs data structure: discontinuous data
https://mne.tools/stable/auto_tutorials/epochs/10_epochs_overview.html

Visualizing epoched data
https://mne.tools/stable/auto_tutorials/epochs/20_visualize_epochs.html

Working with Epoch metadata
https://mne.tools/stable/auto_tutorials/epochs/30_epochs_metadata.html

In [ ]:
tmin = -0.25 #300 ms before the event
tmax = 0.8 #500 ms after the onset

epochs = mne.Epochs(filter_raw,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    preload=True)
epochs

## Autoreject Bad Channels

Handling bad channels
https://mne.tools/stable/auto_tutorials/preprocessing/15_handling_bad_channels.html

Autoreject module
https://autoreject.github.io/stable/index.html

In [ ]:
raw.info['bads'] = []

In [ ]:
picks = mne.pick_types(epochs.info, eeg=True,
                       stim=False, eog=False,
                       include=[], exclude=[])

In [ ]:
print('######################## Computing autoreject. ########################')

In [ ]:
ransac = Ransac(verbose=True, picks=picks, n_jobs=1)
epochs_clean = ransac.fit_transform(epochs)

### Channels List Report

In [ ]:
eeg_path = "eeg"
subname = "sub-"+subject
sesname = "ses-"+session
epoch_reports = "02_epochs"

In [ ]:
#Makes EEG_OUTPUTS -> subject -> session -> eeg -> rawreports -> epoch_reports
if not os.path.exists(os.path.join(output_path, subname, sesname, 
                                   eeg_path, epoch_reports)):
    os.makedirs(os.path.join(output_path, subname, sesname, 
                             eeg_path, epoch_reports))

In [ ]:
epoch_report_path = os.path.join(root_path, "EGI_OUTPUTS\\sub-"+subject+"\\ses-"+session+"\\eeg\\"+epoch_reports)

In [ ]:
bad_channel_title =  subname+"_"+sesname+"_task-"+task+'_bad_channel_report.html'
bad_channel_report = mne.Report(title='Bad Channel Report')
bad_channel_report.add_html(html=(ransac.bad_chs_), title='bad_channel_report.html')
bad_channel_path = os.path.join(epoch_report_path, bad_channel_title)

bad_channel_report.save(bad_channel_path, overwrite=True)

### Ransac Plot Report

In [ ]:
evoked = epochs.average()
evoked_clean = epochs_clean.average()

In [ ]:
evoked.info['bads'] = (ransac.bad_chs_)

In [ ]:
set_matplotlib_defaults(plt)

fig, axes = plt.subplots(2, 1, figsize=(6, 6))

for ax in axes:
    ax.tick_params(axis='x', which='both', bottom='off', top='off')
    ax.tick_params(axis='y', which='both', left='off', right='off')

ylim = dict(eeg=(-40, 50))
evoked.pick_types(eeg=True, exclude=[])
evoked.plot(exclude=[], axes=axes[0], ylim=ylim, show=False)
axes[0].set_title('Before RANSAC', fontsize=12)
evoked_clean.pick_types(eeg=True, exclude=[])
evoked_clean.plot(exclude=[], axes=axes[1], ylim=ylim)
axes[1].set_title('After RANSAC', fontsize=12)
fig.tight_layout()

In [ ]:
ransac_title =  subname+"_"+sesname+"_task-"+task+'_ransac_report.html'
ransac_report = mne.Report(title='Ransac Report')
ransac_report.add_figure(fig=fig, title='ransac_report.html')
ransac_path = os.path.join(epoch_report_path, ransac_title)

ransac_report.save(ransac_path, overwrite=True)

## Bad Channels List

In [ ]:
bad_list = (ransac.bad_chs_)
bad_list

## Save Epochs 

Rename each time to match new subject information.

In [ ]:
epochname = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_epo.fif'
epochname

In [ ]:
epoch_path = os.path.join(epoch_report_path, epochname)

In [ ]:
epochs.save(epoch_path, overwrite=True)